In [1]:
!pip install cdsapi

import os
with open(f'{os.environ.get("HOME")}/.cdsapirc', 'w') as f:
    f.write('url: https://cds.climate.copernicus.eu/api\n')
    f.write('key: ec513420-ea33-41a4-a0be-4f251380c2ca\n')

In [2]:
!pip install pandas xarray numpy matplotlib
!pip install cfgrib netcdf4
!apt-get update && apt-get install -y libeccodes-dev

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 99.7 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ge

In [3]:
# ============================
# CONFIGURACIÓN GENERAL
# ============================
regions = {
    'Sudamérica':    [15.0, -85.0, -60.0, -30.0],     # [N, W, S, E]
    'Centroamérica': [23.0, -98.0,   6.0, -77.0],
    'Europa':        [72.0, -25.0,  34.0,  45.0]
}

years       = list(range(1981, 2025))
months      = [f'{m:02d}' for m in range(1, 13)]
base_period = (1981, 2010)

download_filename = f'era5_multi_vars_{years[0]}_{years[-1]}.grib'
ncfile            = f'era5_multi_vars_{years[0]}_{years[-1]}.nc'

vars_info = [
    {
        'variable_request': '2m_temperature',
        'variable_ds':       't2m',
        'unit_conversion':   lambda da: da - 273.15,
        'unit_label':        '°C'
    },
    {
        'variable_request': 'total_precipitation',
        'variable_ds':       'tp',
        'unit_conversion':   lambda da: da * 1000,   # de m a mm
        'unit_label':        'mm'
    },
    {
        'variable_request': 'skin_temperature',
        'variable_ds':       'skt',
        'unit_conversion':   lambda da: da - 273.15,
        'unit_label':        '°C'
    },
    {
        'variable_request': 'surface_solar_radiation_downwards',
        'variable_ds':       'ssdr',
        'unit_conversion':   None,                    # se deja en J m-2 (puedes convertir si lo deseas)
        'unit_label':        'J m-2'
    }
]


In [4]:
import cdsapi
import os
import xarray as xr # Added this import

plot_dir = "plots_era5_regions"
os.makedirs(plot_dir, exist_ok=True)

# ============================
# DESCARGA ÚNICA DE DATOS (VARIABLES MÚLTIPLES)
# ============================
print("Iniciando descarga de múltiples variables …")
c = cdsapi.Client()
c.retrieve(
    'reanalysis-era5-single-levels-monthly-means',
    {
        'product_type': 'monthly_averaged_reanalysis',
        'variable':         [v['variable_request'] for v in vars_info],
        'year':             [str(y) for y in years],
        'month':            months,
        'time':             '00:00',
        'area':             [90, -180, -90, 180],   # cobertura global para luego recorte por región
        'format':           'grib'
    },
    download_filename
)
print("Descarga completada:", download_filename)

if not os.path.exists(download_filename):
    raise FileNotFoundError(f"No se encontró el archivo de descarga {download_filename}")

# Conversión a NetCDF
if not os.path.exists(ncfile):
    print(f"Convirtiendo a NetCDF: {download_filename} → {ncfile}")
    ds_grib = xr.open_dataset(download_filename, engine='cfgrib', backend_kwargs={'indexpath': ''})
    ds_grib.to_netcdf(ncfile)
    print("Conversión completada.")
else:
    print(f"Ya existe archivo NetCDF: {ncfile}, saltando conversión.")

# Abrir archivo NetCDF
ds = xr.open_dataset(ncfile, engine='netcdf4')
print("Variables disponibles en el dataset:", list(ds.variables))

Iniciando descarga de múltiples variables …


2025-11-09 02:53:27,812 INFO Request ID is 8685c902-845a-47d3-a62e-15a5af4e35f6
INFO:ecmwf.datastores.legacy_client:Request ID is 8685c902-845a-47d3-a62e-15a5af4e35f6
2025-11-09 02:53:28,068 INFO status has been updated to accepted
INFO:ecmwf.datastores.legacy_client:status has been updated to accepted
2025-11-09 02:53:50,424 INFO status has been updated to successful
INFO:ecmwf.datastores.legacy_client:status has been updated to successful


a42c8512e31f525a84904aa11a20915.grib:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

Descarga completada: era5_multi_vars_1981_2024.grib
Convirtiendo a NetCDF: era5_multi_vars_1981_2024.grib → era5_multi_vars_1981_2024.nc


ERROR:cfgrib.dataset:skipping variable: paramId==228 shortName='tp'
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/cfgrib/dataset.py", line 726, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/usr/local/lib/python3.12/dist-packages/cfgrib/dataset.py", line 642, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='time' value=Variable(dimensions=('time',), data=array([ 347155200,  349833600,  352252800,  354931200,  357523200,
        360201600,  362793600,  365472000,  368150400,  370742400,
        373420800,  376012800,  378691200,  381369600,  383788800,
        386467200,  389059200,  391737600,  394329600,  397008000,
        399686400,  402278400,  404956800,  407548800,  410227200,
        412905600,  415324800,  418003200,  420595200,  423273600,
        425865600,  428544000,  431222400,  433814400,  436492800,
        439084800,  441763200,  4444

Conversión completada.
Variables disponibles en el dataset: ['t2m', 'skt', 'number', 'time', 'step', 'surface', 'latitude', 'longitude', 'valid_time']


In [5]:
import cdsapi
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import statsmodels.api as sm


# ============================
# ANÁLISIS PARA CADA VARIABLE Y CADA REGIÓN
# ============================
for vinfo in vars_info:
    var_req  = vinfo['variable_request']
    var_ds   = vinfo['variable_ds']
    conv     = vinfo['unit_conversion']
    label    = vinfo['unit_label']

    if var_ds not in ds.variables:
        print(f"¡Advertencia! Variable '{var_ds}' no encontrada. Variables disponibles: {list(ds.variables)}")
        continue

    for region_name, area in regions.items():
        print(f"\n--- Analizando variable '{var_ds}' en región: {region_name} ---")

        # Recortar la zona espacial
        da_region = ds[var_ds].sel(
            latitude  = slice(area[0], area[2]),
            longitude = slice(area[1], area[3])
        )

        # Aplicar conversión de unidades si es necesario
        if conv is not None:
            da_region = conv(da_region)
        da_region.attrs['units'] = label

        # Promedio espacial
        da_mean = da_region.mean(dim=['latitude', 'longitude'])
        ts      = da_mean.to_series()
        ts.index = pd.to_datetime(ts.index)

        # --- ANÁLISIS A: Línea de base ---
        base_ts       = ts[f'{base_period[0]}-01-01':f'{base_period[1]}-12-31']
        annual        = ts.resample('Y').mean()
        annual_base   = base_ts.resample('Y').mean()
        media_base    = annual_base.mean()

        print(f"{region_name} – {var_ds} – Media anual base ({base_period[0]}-{base_period[1]}): {media_base:.3f} {label}")

        plt.figure(figsize=(10,4))
        plt.plot(annual.index.year, annual.values, label='Media anual')
        plt.hlines(media_base, xmin=annual.index.year.min(), xmax=annual.index.year.max(),
                   color='r', linestyles='--', label=f'Media base {base_period[0]}-{base_period[1]}')
        plt.xlabel('Año')
        plt.ylabel(f'{var_ds} [{label}]')
        plt.title(f'{var_ds} – {region_name} – Serie anual')
        plt.legend()
        plt.grid()
        fname = os.path.join(plot_dir, f"{var_ds}_{region_name}_timeseries_annual.png".replace(" ", "_"))
        plt.savefig(fname, dpi=300, bbox_inches='tight')
        plt.close()
        print("Guardado:", fname)

        # --- ANÁLISIS B: Anomalías & extremos ---
        monthly_base = base_ts.groupby(base_ts.index.month).mean()
        monthly_map  = ts.index.to_series().dt.month.map(monthly_base)
        anomalies    = ts - monthly_map

        plt.figure(figsize=(10,4))
        plt.plot(anomalies.index, anomalies.values, label='Anomalía mensual')
        plt.axhline(0, color='k', linewidth=0.8)
        plt.xlabel('Fecha')
        plt.ylabel(f'Anomalía [{label}]')
        plt.title(f'{var_ds} – {region_name} – Anomalías mensuales')
        plt.legend()
        plt.grid()
        fname = os.path.join(plot_dir, f"{var_ds}_{region_name}_anomalies_monthly.png".replace(" ", "_"))
        plt.savefig(fname, dpi=300, bbox_inches='tight')
        plt.close()
        print("Guardado:", fname)

        sigma        = anomalies.std()
        extreme_up   = anomalies[anomalies > +2*sigma]
        extreme_down = anomalies[anomalies < -2*sigma]
        print(f"{region_name} – {var_ds} – Meses con anomalía alta (> +2σ): {list(extreme_up.index)}")
        print(f"{region_name} – {var_ds} – Meses con anomalía baja (< -2σ): {list(extreme_down.index)}")

        # --- ANÁLISIS C: Impactos / Proyección ---
        y = annual.values
        x = annual.index.year.values
        X = sm.add_constant(x)
        model = sm.OLS(y, X).fit()
        trend = model.params[1]
        print(f"{region_name} – {var_ds} – Tendencia lineal: {trend:.4f} {label} por año")

        future_years = np.arange(x.max() + 1, x.max() + 11)
        future_vals  = model.params[0] + model.params[1] * future_years

        plt.figure(figsize=(10,4))
        plt.plot(x, y, label='Observado')
        plt.plot(future_years, future_vals, '--', label='Proyección (10 años)')
        plt.xlabel('Año')
        plt.ylabel(f'{var_ds} [{label}]')
        plt.title(f'{var_ds} – {region_name} – Proyección (10 años)')
        plt.legend()
        plt.grid()
        fname = os.path.join(plot_dir, f"{var_ds}_{region_name}_projection.png".replace(" ", "_"))
        plt.savefig(fname, dpi=300, bbox_inches='tight')
        plt.close()
        print("Guardado:", fname)

print("\nAnálisis completo para todas variables y regiones.")



--- Analizando variable 't2m' en región: Sudamérica ---


/tmp/ipython-input-449429667.py:44: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  annual        = ts.resample('Y').mean()
/tmp/ipython-input-449429667.py:45: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  annual_base   = base_ts.resample('Y').mean()


Sudamérica – t2m – Media anual base (1981-2010): 17.901 °C
Guardado: plots_era5_regions/t2m_Sudamérica_timeseries_annual.png
Guardado: plots_era5_regions/t2m_Sudamérica_anomalies_monthly.png
Sudamérica – t2m – Meses con anomalía alta (> +2σ): [Timestamp('1998-07-01 00:00:00'), Timestamp('2006-07-01 00:00:00'), Timestamp('2012-09-01 00:00:00'), Timestamp('2015-07-01 00:00:00'), Timestamp('2015-08-01 00:00:00'), Timestamp('2016-02-01 00:00:00'), Timestamp('2017-02-01 00:00:00'), Timestamp('2017-05-01 00:00:00'), Timestamp('2017-07-01 00:00:00'), Timestamp('2017-08-01 00:00:00'), Timestamp('2017-09-01 00:00:00'), Timestamp('2018-09-01 00:00:00'), Timestamp('2019-06-01 00:00:00'), Timestamp('2020-03-01 00:00:00'), Timestamp('2021-09-01 00:00:00'), Timestamp('2023-05-01 00:00:00'), Timestamp('2023-06-01 00:00:00'), Timestamp('2023-07-01 00:00:00'), Timestamp('2023-08-01 00:00:00'), Timestamp('2023-09-01 00:00:00'), Timestamp('2023-10-01 00:00:00'), Timestamp('2023-11-01 00:00:00'), Timestam

/tmp/ipython-input-449429667.py:44: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  annual        = ts.resample('Y').mean()
/tmp/ipython-input-449429667.py:45: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  annual_base   = base_ts.resample('Y').mean()


Centroamérica – t2m – Media anual base (1981-2010): 25.770 °C
Guardado: plots_era5_regions/t2m_Centroamérica_timeseries_annual.png
Guardado: plots_era5_regions/t2m_Centroamérica_anomalies_monthly.png
Centroamérica – t2m – Meses con anomalía alta (> +2σ): [Timestamp('1998-01-01 00:00:00'), Timestamp('1998-02-01 00:00:00'), Timestamp('2003-03-01 00:00:00'), Timestamp('2015-08-01 00:00:00'), Timestamp('2015-09-01 00:00:00'), Timestamp('2015-10-01 00:00:00'), Timestamp('2015-11-01 00:00:00'), Timestamp('2015-12-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-03-01 00:00:00'), Timestamp('2016-05-01 00:00:00'), Timestamp('2016-12-01 00:00:00'), Timestamp('2019-02-01 00:00:00'), Timestamp('2019-06-01 00:00:00'), Timestamp('2019-08-01 00:00:00'), Timestamp('2020-01-01 00:00:00'), Timestamp('2020-02-01 00:00:00'), Timestamp('2020-04-01 00:00:00'), Timestamp('2023-05-01 00:00:00'), Timestamp('2023-06-01 00:00:00'), Timestamp('2023-07-01 00:00:00'), Timestamp('2023-08-01 00:00:00

...

Sudamérica – skt – Media anual base (1981-2010): 18.540 °C
Guardado: plots_era5_regions/skt_Sudamérica_timeseries_annual.png
Guardado: plots_era5_regions/skt_Sudamérica_anomalies_monthly.png
Sudamérica – skt – Meses con anomalía alta (> +2σ): [Timestamp('1998-07-01 00:00:00'), Timestamp('2012-09-01 00:00:00'), Timestamp('2015-09-01 00:00:00'), Timestamp('2017-08-01 00:00:00'), Timestamp('2017-09-01 00:00:00'), Timestamp('2019-06-01 00:00:00'), Timestamp('2019-11-01 00:00:00'), Timestamp('2020-03-01 00:00:00'), Timestamp('2020-10-01 00:00:00'), Timestamp('2021-09-01 00:00:00'), Timestamp('2023-03-01 00:00:00'), Timestamp('2023-05-01 00:00:00'), Timestamp('2023-06-01 00:00:00'), Timestamp('2023-07-01 00:00:00'), Timestamp('2023-08-01 00:00:00'), Timestamp('2023-09-01 00:00:00'), Timestamp('2023-10-01 00:00:00'), Timestamp('2023-11-01 00:00:00'), Timestamp('2023-12-01 00:00:00'), Timestamp('2024-01-01 00:00:00'), Timestamp('2024-02-01 00:00:00'), Timestamp('2024-03-01 00:00:00'), Timestam

/tmp/ipython-input-449429667.py:44: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  annual        = ts.resample('Y').mean()
/tmp/ipython-input-449429667.py:45: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  annual_base   = base_ts.resample('Y').mean()


Guardado: plots_era5_regions/skt_Centroamérica_timeseries_annual.png
Guardado: plots_era5_regions/skt_Centroamérica_anomalies_monthly.png
Centroamérica – skt – Meses con anomalía alta (> +2σ): [Timestamp('1998-01-01 00:00:00'), Timestamp('1998-02-01 00:00:00'), Timestamp('1998-06-01 00:00:00'), Timestamp('2003-03-01 00:00:00'), Timestamp('2009-10-01 00:00:00'), Timestamp('2015-09-01 00:00:00'), Timestamp('2015-10-01 00:00:00'), Timestamp('2015-11-01 00:00:00'), Timestamp('2015-12-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-05-01 00:00:00'), Timestamp('2020-02-01 00:00:00'), Timestamp('2020-04-01 00:00:00'), Timestamp('2021-10-01 00:00:00'), Timestamp('2023-05-01 00:00:00'), Timestamp('2023-06-01 00:00:00'), Timestamp('2023-07-01 00:00:00'), Timestamp('2023-08-01 00:00:00'), Timestamp('2023-09-01 00:00:00'), Timestamp('2023-10-01 00:00:00'), Timestamp('2023-11-01 00:00:00'), Timestamp('2023-12-01 00:00:00'), Timestamp('2024-01-01 00:00:00'), Timestamp('2024-03-01 00

/tmp/ipython-input-449429667.py:44: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  annual        = ts.resample('Y').mean()
/tmp/ipython-input-449429667.py:45: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  annual_base   = base_ts.resample('Y').mean()


Europa – skt – Media anual base (1981-2010): 9.431 °C
Guardado: plots_era5_regions/skt_Europa_timeseries_annual.png
Guardado: plots_era5_regions/skt_Europa_anomalies_monthly.png
Europa – skt – Meses con anomalía alta (> +2σ): [Timestamp('1990-02-01 00:00:00'), Timestamp('1995-02-01 00:00:00'), Timestamp('2002-02-01 00:00:00'), Timestamp('2006-12-01 00:00:00'), Timestamp('2007-01-01 00:00:00'), Timestamp('2008-02-01 00:00:00'), Timestamp('2014-02-01 00:00:00'), Timestamp('2015-12-01 00:00:00'), Timestamp('2016-02-01 00:00:00'), Timestamp('2018-05-01 00:00:00'), Timestamp('2019-12-01 00:00:00'), Timestamp('2020-01-01 00:00:00'), Timestamp('2020-02-01 00:00:00'), Timestamp('2020-11-01 00:00:00'), Timestamp('2022-02-01 00:00:00'), Timestamp('2022-08-01 00:00:00'), Timestamp('2022-11-01 00:00:00'), Timestamp('2023-01-01 00:00:00'), Timestamp('2023-09-01 00:00:00'), Timestamp('2024-02-01 00:00:00'), Timestamp('2024-03-01 00:00:00'), Timestamp('2024-06-01 00:00:00'), Timestamp('2024-08-01 00: